In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

os.environ["TAVILY_API_KEY"] = ""

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()

In [4]:
query = "백터기반의 검색을 할때 유클리드 와 코사인의 차이점은?"

tool.invoke({"query": query})

[{'url': 'https://only-wanna.tistory.com/entry/Cosine-Similarity코사인-유사도와-Euclidean-Distance유클리드-거리-알아보기',
  'content': 'Cosine 유사도 - 유클리드 거리 사이 관계. 코사인 유사도와 유클리드 거리는 연관이 거의 없어 보입니다. 하지만 수식적으로 정리하면, 둘 사이 관계를 다음과 같이 표현할 수 있습니다. 유클리드 거리의 제곱을 정리하면 다음과 같게 되고. 위의 식은 다음의 식과 ...'},
 {'url': 'https://docs.likejazz.com/cosine-sim/',
  'content': '코사인 유사도의 의미. 01 Jun 2018. 일반적으로 문서간의 유사도를 비교할 때는 코사인 유사도 cosine similarity 를 주로 사용한다. 여기서는 그 동안 습관적으로 사용해오던 코사인 유사도의 의미를 수식과 함께 예제를 통해 살펴보도록 한다. 2018년 6월 1일 초안 작성 ...'},
 {'url': 'https://techscene.tistory.com/entry/머신러닝-유사도-및-거리-총정리-코사인-유클리디안-자카드-멘하탄-알고리즘',
  'content': '안녕하세요, 테크씬입니다. 데이터 분석과 머신러닝 분야에서 유사도와 거리 측정은 매우 중요한 역할을 합니다. 이러한 측정 방법은 다양한 데이터 유형과 문제 상황에 따라 그 중요성이 달라질 수 있으며, 다양한 유사도 및 거리 측정 방법은 각각 고유의 특성과 적용 분야를 가지고 있습니다. 이 ...'},
 {'url': 'https://codong.tistory.com/35',
  'content': '2.1 유클리드 거리(Euclidean distance) 유클리드 거리는 문서 유사도를 구할 때 자카드 유사도나 코사인 유사도만큼 유용한 방법은 아니다. 하지만 여러 가지 방법을 이해하고, 시도해보는 것 자체만으로 다른 개념들을 이해할 때 도움이 되므로 의미가 있다.'},
 {'url': '

In [5]:
from langchain.retrievers import (
    ContextualCompressionRetriever,
    TavilySearchAPIRetriever,
)

base_tavily_retriever = TavilySearchAPIRetriever(
	k=6, include_raw_content=True, include_images=True
)

# tavily_retriever = ContextualCompressionRetriever(
# 	base_compressor=pipeline_compressor, base_retriever=base_tavily_retriever
# )

In [7]:
docs = base_tavily_retriever.get_relevant_documents(query)

In [9]:
for doc in docs:
    print(doc.metadata)
    print(doc.page_content)
    print("-----------------------------------")

{'title': '[딥러닝 Nlp] 05. 벡터의 유사도(코사인, 유클리드, 자카드)', 'source': 'https://222ys.tistory.com/17', 'score': 0.96354, 'images': ['https://i.ytimg.com/vi/-4qequgou6E/maxresdefault.jpg', 'https://i.ytimg.com/vi/O-Ns2ADCsPg/maxresdefault.jpg', 'https://i.ytimg.com/vi/aXosYpzK90k/maxresdefault.jpg', 'https://s3.orbi.kr/data/file/united2/ce2fc18d9f1e424c969019d7dce26522.png', 'https://w1.pngwing.com/pngs/322/710/png-transparent-unit-circle-trigonometric-functions-law-of-cosines-trigonometry-law-of-sines-angle-inverse-trigonometric-functions-mathematics.png']}
코딩빌런 솜지
POWERED BY TISTORY
[딥러닝 NLP] 05. 벡터의 유사도(코사인, 유클리드, 자카드)
문서의 유사도 : 문서들 간에 동일하거나 비슷한 단어가 공통적으로 사용된 정도
성능: 단어 표현 방법(DTM, Word2Vec 등) 과 유사도 기법(유클리드 거리, 코사인 유사도 등) 에 따라 달라짐
# 05-01 코사인 유사도(Cosine Similarity)
1. 코사인 유사도란?
-두 벡터가 가리키는 방향이 유사한 정도를 코사인 각도로  구하는 기법
-동일 = 1, 90° = 0, 180° = -1 이므로, -1 이상 1 이하이며  1에 가까울수록 유사
-특징: 문서의 길이가 다른 상황에서 비교적 공정한 비교 가능
Ex) 문서A, 문서B는 내용 비슷한데 길이는 2배 차이 / 문서A, 문서C는 다른 내용인데 길이는 비슷한 경우
유클리드 거리는 길이의 영향 받아 A와 C의

In [12]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.document_compressors import (
    DocumentCompressorPipeline,
    EmbeddingsFilter,
)
from langchain.retrievers import (
    ContextualCompressionRetriever,
    TavilySearchAPIRetriever,
)

embeddings = OpenAIEmbeddings()
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=20)
relevance_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8)
pipeline_compressor = DocumentCompressorPipeline(
	transformers=[splitter, relevance_filter]
)
base_tavily_retriever = TavilySearchAPIRetriever(
	k=6, include_raw_content=True, include_images=True
)

tavily_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=base_tavily_retriever
)

In [13]:
docs = tavily_retriever.get_relevant_documents(query)

for doc in docs:
    print(doc.metadata)
    print(doc.page_content)
    print("-----------------------------------")

{'title': '벡터 유사도 검색이 무엇인가요? (What is Vector Similarity Search?)', 'source': 'https://discuss.pytorch.kr/t/what-is-vector-similarity-search/2475', 'score': 0.97289, 'images': ['https://i.ytimg.com/vi/-4qequgou6E/maxresdefault.jpg', 'https://i.ytimg.com/vi/O-Ns2ADCsPg/maxresdefault.jpg', 'https://i.ytimg.com/vi/aXosYpzK90k/maxresdefault.jpg', 'https://s3.orbi.kr/data/file/united2/ce2fc18d9f1e424c969019d7dce26522.png', 'https://w1.pngwing.com/pngs/322/710/png-transparent-unit-circle-trigonometric-functions-law-of-cosines-trigonometry-law-of-sines-angle-inverse-trigonometric-functions-mathematics.png']}
확장성 / Scalability
대규모 데이터셋을 검색하는데는 연산 비용과 시간이 많이 소요될 수 있습니다. 벡터 유사도 검색 알고리즘과 데이터 구조는 검색 공간을 효율적으로 정리(prune)하여, 거리 연산 횟수를 줄이고 유사한 항목들을 더 빠르게 검색할 수 있는 방법을 제공합니다.
Searching through large datasets can be computationally expensive and time-consuming. Vector similarity search algorithms and data structures provide efficient ways to prune the search space, reducing the number of distance computat

In [4]:
RESPONSE_TEMPLATE = """\
You are an expert researcher and writer, tasked with answering any question.

Generate a comprehensive and informative, yet concise answer of 250 words or less for the \
given question based solely on the provided search results (URL and content). You must \
only use information from the provided search results. Use an unbiased and \
journalistic tone. Combine search results together into a coherent answer. Do not \
repeat text. Cite search results using [${{number}}] notation. Only cite the most \
relevant results that answer the question accurately. Place these citations at the end \
of the sentence or paragraph that reference them - do not put them all at the end. If \
different results refer to different entities within the same name, write separate \
answers for each entity. If you want to cite multiple results for the same sentence, \
format it as `[${{number1}}] [${{number2}}]`. However, you should NEVER do this with the \
same number - if you want to cite `number1` multiple times for a sentence, only do \
`[${{number1}}]` not `[${{number1}}] [${{number1}}]`

You should use bullet points in your answer for readability. Put citations where they apply \
rather than putting them all at the end.

If there is nothing in the context relevant to the question at hand, just say "Hmm, \
I'm not sure." Don't try to make up an answer.

Anything between the following `context` html blocks is retrieved from a knowledge \
bank, not part of the conversation with the user.

<context>
    {context}
<context/>

REMEMBER: If there is no relevant information within the context, just say "Hmm, I'm \
not sure." Don't try to make up an answer. Anything between the preceding 'context' \
html blocks is retrieved from a knowledge bank, not part of the conversation with the \
user. The current date is {current_date}.
"""

In [5]:
REPHRASE_TEMPLATE = """\
Given the following conversation and a follow up question, rephrase the follow up \
question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone Question:"""

In [6]:
import asyncio
from typing import List, Optional, Sequence, Tuple, Union
from datetime import datetime
from operator import itemgetter

from langchain.schema import Document
from langchain.schema.language_model import BaseLanguageModel
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.retriever import BaseRetriever
from langchain.schema.runnable import (
    ConfigurableField,
    Runnable,
    RunnableBranch,
    RunnableLambda,
    RunnableMap,
)

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from pydantic import BaseModel, Field
from langchain.schema.messages import AIMessage, HumanMessage

In [7]:
class ChatRequest(BaseModel):
    question: str
    chat_history: List[Tuple[str, str]] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "question", "output": "answer"}},
    )


def serialize_history(request: ChatRequest):
    chat_history = request.get("chat_history", [])
    converted_chat_history = []
    for message in chat_history:
        if message[0] == "human":
            converted_chat_history.append(HumanMessage(content=message[1]))
        elif message[0] == "ai":
            converted_chat_history.append(AIMessage(content=message[1]))
    return converted_chat_history

In [8]:
def create_retriever_chain(
    llm: BaseLanguageModel, retriever: BaseRetriever
) -> Runnable:
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(REPHRASE_TEMPLATE)
    condense_question_chain = (
        CONDENSE_QUESTION_PROMPT | llm | StrOutputParser()
    ).with_config(
        run_name="CondenseQuestion",
    )
    conversation_chain = condense_question_chain | retriever
    return RunnableBranch(
        (
            RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
                run_name="HasChatHistoryCheck"
            ),
            conversation_chain.with_config(run_name="RetrievalChainWithHistory"),
        ),
        (
            RunnableLambda(itemgetter("question")).with_config(
                run_name="Itemgetter:question"
            )
            | retriever
        ).with_config(run_name="RetrievalChainWithNoHistory"),
    ).with_config(run_name="RouteDependingOnChatHistory")

In [9]:
def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = f"<doc id='{i}'>{doc.page_content}</doc>"
        formatted_docs.append(doc_string)
    return "\n".join(formatted_docs)

def create_chain(
    llm: BaseLanguageModel,
    retriever: BaseRetriever,
) -> Runnable:
    retriever_chain = create_retriever_chain(llm, retriever) | RunnableLambda(
        format_docs
    ).with_config(run_name="FormatDocumentChunks")
    _context = RunnableMap(
        {
            "context": retriever_chain.with_config(run_name="RetrievalChain"),
            "question": RunnableLambda(itemgetter("question")).with_config(
                run_name="Itemgetter:question"
            ),
            "chat_history": RunnableLambda(itemgetter("chat_history")).with_config(
                run_name="Itemgetter:chat_history"
            ),
        }
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", RESPONSE_TEMPLATE),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    ).partial(current_date=datetime.now().isoformat())

    response_synthesizer = (prompt | llm | StrOutputParser()).with_config(
        run_name="GenerateResponse",
    )
    return (
        {
            "question": RunnableLambda(itemgetter("question")).with_config(
                run_name="Itemgetter:question"
            ),
            "chat_history": RunnableLambda(serialize_history).with_config(
                run_name="SerializeHistory"
            ),
        }
        | _context
        | response_synthesizer
    )

In [13]:
# from langchain.chat_models import ChatAnthropic, ChatOpenAI, ChatVertexAI
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="gpt-3.5-turbo-16k",
#     # model="gpt-4",
#     streaming=True,
#     temperature=0.1,
# )
# retriever = tavily_retriever

# llm = ChatOpenAI(
#     base_url="http://localhost:1234/v1",
#     temperature=0,
#     api_key="lm-studio",
#     streaming=True,
# )

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    # model="gpt-4",
    streaming=True,
    temperature=0.1,
)

chain = create_chain(llm, tavily_retriever)

In [20]:
query = "tanstack/react-query에서 suspense 사용하는 방법을 알려줘"

result = chain.invoke({"question": query})

In [21]:
print(result)

React Query에서 suspense를 사용하는 방법은 다음과 같습니다:

1. React Query의 Suspense 모드를 활성화하려면 전역 또는 쿼리 수준의 설정에서 suspense 옵션을 true로 설정해야 합니다. [1]
   - 전역 설정: `setQueryClientOptions({ suspense: true })`
   - 쿼리 수준 설정: `useQuery(key, queryFn, { suspense: true })`

2. Suspense 모드를 사용하면 쿼리 컴포넌트가 마운트되기 전에 데이터를 미리 로드할 수 있습니다. 이를 위해 라우팅 콜백이나 사용자 상호작용 이벤트에서 Prefetching을 구현하는 것이 좋습니다. [0]

3. Suspense 모드에서는 상태 및 오류 객체 대신 React.Suspense 컴포넌트를 사용하여 상태 및 오류를 처리합니다. 오류 처리를 위해 React 에러 경계를 사용할 수도 있습니다. [3]

4. Suspense 모드에서는 쿼리 오류가 발생하면 오류가 컴포넌트의 다음 렌더링에서 발생하고 가장 가까운 오류 경계로 전파됩니다. 이 동작을 비활성화하려면 useErrorBoundary 옵션을 false로 설정하면 됩니다. 오류가 전혀 발생하지 않도록 하려면 throwOnError 옵션을 false로 설정할 수도 있습니다. [2]

5. Suspense 모드에서는 쿼리 컴포넌트의 마운트 시점에 쿼리를 가져오고 일시 중단할 수 있습니다. 이는 Fetch-on-render 솔루션으로 작동하며 추가 구성 없이 사용할 수 있습니다. [0]

더 자세한 내용은 React Query의 공식 문서와 커뮤니티 리소스를 참조하시기 바랍니다.
